In [2]:
# ====================================================
#  Logiciel de décision d'investissement - Bourse & IA
# ====================================================

import tkinter as tk
from tkinter import ttk
import pandas as pd
import numpy as np
from datetime import datetime
import yfinance as yf
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV
from sklearn.metrics import mean_squared_error
import statsmodels.api as sm


# =============================
# Analyse de l'actualité (sentiment)
# =============================

def analyse_sentiments(ticker):
    try:
        df_news = pd.read_csv("news_articles_2025-05-06.csv")
        df_ticker = df_news[df_news['ticker'] == ticker]
        if df_ticker.empty:
            return "neutre", 0.0
        score = df_ticker["sentiment_score"].mean()
        if score > 0.1:
            return "positive", score
        elif score < -0.1:
            return "negative", score
        else:
            return "neutre", score
    except:
        return "neutre", 0.0


# =============================
# Analyse technique - indicateurs
# =============================

def compute_indicators(df):
    df['sma20'] = df['close'].rolling(window=20).mean()
    df['ema12'] = df['close'].ewm(span=12, adjust=False).mean()
    delta = df['close'].diff()
    gain = np.where(delta > 0, delta, 0)
    loss = np.where(delta < 0, -delta, 0)
    avg_gain = pd.Series(gain).rolling(window=14).mean()
    avg_loss = pd.Series(loss).rolling(window=14).mean()
    rs = avg_gain / (avg_loss + 1e-10)
    df['rsi14'] = 100 - (100 / (1 + rs))
    df['ema26'] = df['close'].ewm(span=26, adjust=False).mean()
    df['macd'] = df['ema12'] - df['ema26']
    df['std20'] = df['close'].rolling(window=20).std()
    df['upperband'] = df['sma20'] + 2 * df['std20']
    df['lowerband'] = df['sma20'] - 2 * df['std20']
    df['percentb'] = (df['close'] - df['lowerband']) / (df['upperband'] - df['lowerband'] + 1e-10)
    df['roc'] = df['close'].pct_change(periods=14)
    return df


# =============================
# Scoring des indicateurs
# =============================

def score_rsi(rsi):
    if rsi < 30:
        return 1.0
    elif rsi > 70:
        return 0.0
    else:
        return (70 - rsi) / 40

def score_macd(macd):
    return 1.0 if macd > 0 else 0.0

def score_sma(close, sma):
    return 1.0 if close > sma else 0.0

def score_bbands(percentb):
    if percentb < 0.2:
        return 1.0
    elif percentb > 0.8:
        return 0.0
    else:
        return 1 - ((percentb - 0.2) / 0.6)

def score_roc(roc):
    return 1.0 if roc > 0 else 0.0

def calculate_technical_score(latest_data):
    return sum([
        score_rsi(latest_data['rsi14']),
        score_macd(latest_data['macd']),
        score_sma(latest_data['close'], latest_data['sma20']),
        score_bbands(latest_data['percentb']),
        score_roc(latest_data['roc'])
    ])


# =============================
# Préparation pour la prédiction
# =============================

def prepare_features(df):
    df_feat = df.copy()
    df_feat['target'] = df_feat['close'].shift(-1)
    df_feat.dropna(inplace=True)
    features = ['open', 'high', 'low', 'close', 'volume',
                'sma20', 'ema12', 'rsi14', 'macd', 'percentb', 'roc']
    return df_feat[features], df_feat['target']

def train_test_split_time_series(X, y, test_size=0.2):
    split_index = int(len(X) * (1 - test_size))
    return X.iloc[:split_index], X.iloc[split_index:], y.iloc[:split_index], y.iloc[split_index:]


# =============================
# Modèles de prédiction
# =============================

def model_linear_regression(X_train, y_train, X_test):
    model = LinearRegression().fit(X_train, y_train)
    return model.predict(X_test), model

def model_random_forest(X_train, y_train, X_test):
    grid = GridSearchCV(RandomForestRegressor(random_state=42),
                        {'n_estimators': [50, 100], 'max_depth': [3, 5, 7]},
                        cv=TimeSeriesSplit(n_splits=5),
                        scoring='neg_mean_squared_error')
    grid.fit(X_train, y_train)
    return grid.best_estimator_.predict(X_test), grid.best_estimator_

def model_sarimax(y_train, y_test):
    best_model, best_aic = None, float('inf')
    for order in [(1,1,1), (2,1,2), (1,0,1), (2,0,2)]:
        try:
            model = sm.tsa.statespace.SARIMAX(y_train, order=order, enforce_stationarity=False, enforce_invertibility=False)
            fitted = model.fit(disp=False)
            if fitted.aic < best_aic:
                best_aic, best_model = fitted.aic, fitted
        except:
            continue
    return best_model.predict(start=y_test.index[0], end=y_test.index[-1]), best_model


# =============================
# Analyse globale (sentiment + technique)
# =============================

def analyse_globale(ticker):
    try:
        df = yf.download(ticker, start="2020-01-01", end=datetime.today().strftime('%Y-%m-%d'))
        if df.empty:
            return "Erreur : données indisponibles", 0.0
        df.reset_index(inplace=True)
        df.columns = [col[1].lower() if isinstance(col, tuple) else col.lower() for col in df.columns]
    except Exception as e:
        return f"Erreur récupération données : {e}", 0.0

    df.sort_values("date", inplace=True)
    df = compute_indicators(df)
    latest = df.iloc[-1]
    tech_score = calculate_technical_score(latest)

    X, y = prepare_features(df)
    X_train, X_test, y_train, y_test = train_test_split_time_series(X, y)

    preds = {}
    preds['LinReg'], lin_model = model_linear_regression(X_train, y_train, X_test)
    preds['RF'], rf_model = model_random_forest(X_train, y_train, X_test)
    preds['SARIMAX'], sarimax_model = model_sarimax(y_train, y_test)

    mses = {model: mean_squared_error(y_test, preds[model]) for model in preds}
    best_model = min(mses, key=mses.get)

    if best_model == 'LinReg':
        prediction = lin_model.predict(X.iloc[[-1]])[0]
    elif best_model == 'RF':
        prediction = rf_model.predict(X.iloc[[-1]])[0]
    else:
        prediction = sarimax_model.forecast(steps=1)[0]

    last_close = df.iloc[-1]['close']
    decision = "Investir" if tech_score > 3 and prediction > last_close else "Ne pas investir"
    return decision, round(prediction, 2)


# =============================
# Interface graphique (Tkinter)
# =============================

class InvestmentApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Investissement Bourse - IA")
        self.ticker_selected = tk.StringVar()
        self.result_text = tk.StringVar()
        self.build_ui()

    def build_ui(self):
        tk.Label(self.root, text="Investissement Bourse - IA", font=("Arial", 18, "bold")).pack(pady=10)

        tk.Label(
            self.root,
            text="Ce logiciel vous permet de prendre une décision d'investissement sur une sélection d'actions\n"
                 "à partir d'une analyse de l'actualité puis d'une analyse technique avec prédictions de machine learning.",
            font=("Arial", 11), justify="center"
        ).pack(pady=10)

        frame = tk.Frame(self.root)
        frame.pack(pady=10)

        tk.Label(frame, text="Sélectionnez une action :", font=("Arial", 12)).pack(side="left", padx=5)

        self.ticker_combo = ttk.Combobox(frame, textvariable=self.ticker_selected, state="readonly")
        self.ticker_combo['values'] = ['AAPL', 'MSFT', 'TSLA', 'AMZN', 'GOOG']
        self.ticker_combo.pack(side="left", padx=5)

        self.analyse_btn = tk.Button(self.root, text="Lancer l'analyse", command=self.lancer_analyse, state="disabled")
        self.analyse_btn.pack(pady=10)

        tk.Label(self.root, textvariable=self.result_text, font=("Arial", 13, "bold"), fg="blue").pack(pady=10)

        self.ticker_combo.bind("<<ComboboxSelected>>", lambda e: self.analyse_btn.config(state="normal"))

    def lancer_analyse(self):
        ticker = self.ticker_selected.get()
        sentiment_decision, _ = analyse_sentiments(ticker)
        technique_decision, predicted = analyse_globale(ticker)
        self.result_text.set(f"{technique_decision} : {predicted}")


# =============================
# Lancement de l'application
# =============================

if __name__ == "__main__":
    root = tk.Tk()
    app = InvestmentApp(root)
    root.geometry("750x400")
    root.mainloop()

[*********************100%***********************]  1 of 1 completed

1 Failed download:
['TSLA']: YFRateLimitError('Too Many Requests. Rate limited. Try after a while.')


: 